# **Creating a Global Friction Surface Using QGIS**

## Intro

This guide will take you through the process of creating a global friction surface for estimating travel times to cities in the 1980s. The goal is to integrate various raster data sources—such as urbanization, shipping lanes, topography, country borders, road networks, and more—into a cohesive friction surface that models the travel time to the nearest urban center from any given location.

To create the desired friction surface, we identified eight key features that have the greatest impact on travel time. These include:[]. After pinpointing these features, we gathered raster data corresponding to the necessary information and transformed each into a distinct friction layer. Below is a table listing all the datasets used in this guide along with their respective uses.


| Use                     | Description                                      | Link                                                                 |
|-------------------------|--------------------------------------------------|----------------------------------------------------------------------|
| Border Crossing Time    | Shapefile of all country borders                 | [Link](https://github.com/nvkelso/natural-earth-vector/blob/master/50m_cultural/ne_50m_admin_0_boundary_lines_land.shp) |
| Border Crossing Time    | Shapefile of all Continents                      | [Link](Continent_raster.tif)                                         |
| World Map               | World Map                                        | [Link](https://international.ipums.org/international/gis.shtml)      |
| Port Data               | Port Data                                        | [Link](https://fgmod.nga.mil/apps/WPI-Viewer/)                       |
| Road Network            | Road Network                                     | [Link](https://hgl.harvard.edu/catalog/harvard-dcw-rd-line)          |
| Railway Network         | Railway Network                                  | [Link](https://hgl.harvard.edu/catalog/harvard-dcw-rr-line)          |
| Navigable Rivers        | Navigable Rivers                                 | [Link](http://gaia.geosci.unc.edu/rivers/)                           |
| Major Water Bodies      | Major Water Bodies                               | [Link](https://www.worldwildlife.org/pages/global-lakes-and-wetlands-database) |
| Shipping Lanes          | Shipping Lanes                                   | [Link](https://purl.stanford.edu/zv735jp7944)                        |
| National Borders (Current) | National Borders (Current)                   | [Link](https://github.com/nvkelso/natural-earth-vector/blob/master/50m_cultural/ne_50m_admin_0_boundary_lines_land.shp) |
| Degree of Urbanization  | Degree of Urbanization                           | [Link](https://jeodpp.jrc.ec.europa.eu/ftp/jrc-opendata/GHSL/GHS_SMOD_GLOBE_R2023A/GHS_SMOD_E1980_GLOBE_R2023A_54009_1000/V1-0/) |
| Land Cover              | Land Cover                                       | [Link](https://forobs.jrc.ec.europa.eu/glc2000/data)                 |
| Elevation               | Elevation                                        | [Link](https://www.usgs.gov/centers/eros/science/usgs-eros-archive-digital-elevation-global-30-arc-second-elevation-gtopo30?qt-science_center_objects=0#qt-science_center_objects) |



# **Part 1: Upload World Map**

### Step 1: Upload the World Map shapefile 
**Objective**: Load the shapefile for the World Map  into QGIS.
Open QGIS, then use the **'Add Vector Layer'** tool to load the shapefile containing the country borders.

### Step 2: Reproject the World Map Vector
 **Objective**: Reproject the World Map vector layer into the WGS 84 projection (EPSG:4326).
 Open the **'Reproject Layer'** tool from the Processing Toolbox.
 In the **Input Layer** field, select the World Map vector.
 In the **Target CRS**, choose **EPSG:4326 - WGS 84** to ensure consistency with the Mercator projection you're using.
Click ‘Run’ 


---
---

# **Part 2: Make 1km x 1km grid**




### Step 1: Create 1km x 1km grid
**Objective**: 
QGIS Toolbox: search “Create Grid”
Grid Type: Rectangle
Grid Extent: World Map

processing.run("native:creategrid", {'TYPE':2,'EXTENT':'-180.000000000,180.000000000,-89.999983420,83.633392334 [EPSG:4326]','HSPACING':0.033333,'VSPACING':0.033333,'HOVERLAY':0,'VOVERLAY':0,'CRS':QgsCoordinateReferenceSystem('EPSG:4326'),'OUTPUT':'TEMPORARY_OUTPUT'})


---
---

# **Part 3: Individual Trasformations of the Ratser Data**

# 1. Border Crossing Time Calculation

This guide outlines how to calculate border crossing times by assigning friction weights to borders based on continent-level analysis. We will reproject the continent raster and rasterize the country borders to represent the time it takes to cross borders. This simplified approach helps to account for over 80 countries efficiently.

## Step-by-Step Process

### Step 1: Open Shapefiles
- **Objective**: Load the shapefile for all country borders and all continents into QGIS.
  - Open QGIS, then use the **'Add Vector Layer'** tool to load the shapefile containing the country borders.
  - Similarly, load the shapefile containing the continents layer.

### Step 2: Reproject the Continent Raster
- **Objective**: Reproject the continent raster layer into the WGS 84 projection (EPSG:4326).
  - Open the **'Warp (Reproject)'** tool from the Processing Toolbox.
  - In the **Input Layer** field, select the continent raster.
  - In the **Target CRS**, choose **EPSG:4326 - WGS 84** to ensure consistency with the Mercator projection you're using.

### Step 3: Convert Country Borders to Raster
- **Objective**: Convert the country borders vector layer (line geometry) into a raster format.
  - Open the **'Rasterize (Vector to Raster)'** tool from the Processing Toolbox.
  - In the **Input Layer**, select the country borders layer.
  - Set both **Width/Horizontal resolution** and **Height/Vertical resolution** to **0.00833** to create a raster with appropriate resolution.
  - This step converts vector country boundaries into a grid format, where each pixel will represent a specific area.

### Step 4: Fill Null Values in Raster
- **Objective**: Ensure all NULL values in the raster are filled appropriately.
  - Open the **'Fill Data'** tool from the Processing Toolbox.
  - In the **Input Layer** select the newly made raster border file
  - Set the **Fill Value** attribute to **0** 

### Step 5: Modify Border Raster with a Distinct Values
- **Objective**: Assign a new value to the modified border raster to help with later calculations.
  - In the **Input Layer** select the newly filled raster border file
  - Open the **'Raster Calculator'** from the Processing Toolbox.
  - In the calculator, use the formula:
    ``` 
    ("guide_border_fill@1" = 1) * 0.5
    ```
    - This assigns a friction weight (0.5) to the areas of the border raster that have value 1, representing the country borders.
  - Save the output as **'halved_modified_borders'** to preserve this modified raster.

### Step 6: Calculate the Difference Between Continent and Border Rasters
- **Objective**: Subtract the modified border raster from the continent raster to identify differences, so that each border has a unqiue value cooresponding to its continent.
  - Open the **'Raster Calculator'** again.
  - Use the formula:
    ``` 
    "Continent_raster@1" - "guide_border_fill@1"
    ```
  - This operation will highlight the differences between the continent raster and the border data, which can then be used to calculate border crossing times.
  - Save the output as **'continent_border_diff'**.

### Step 7: Assign Friction Weights Based on Border Differences
- **Objective**: Assign specific friction weights based on the values in the **'continent_border_diff'** raster.
  - Open the **'Raster Calculator'** once more.
  - Use the following formula to assign specific friction values based on the differences between continents and borders:
    C 
    ("continent_border_diff@1" = 0.5) * 0.115 + 
    ("continent_border_diff@1" = 1.5) * 0.075 + 
    ("continent_border_diff@1" = 2.5) * 0.075 + 
    ("continent_border_diff@1" = 3.5) * 0.06 + 
    ("continent_border_diff@1" = 4.5) * 0.075 + 
    ("continent_border_diff@1" = 5.5) * 0.055 + 
    ("continent_border_diff@1" = 6.5) * 0.075 + 
    ("continent_border_diff@1" = 7.5) * 0.035
    ```
    - This formula assigns different friction weights based on the values found in **'continent_border_diff'**. The weights (e.g., 0.115, 0.075) represent time values (minutes per meter) associated with different levels of border difficulty.

---

### Final Output
By following these steps, you’ll convert your country border and continent shapefiles into a raster format, modify the raster with specific friction values, and calculate border crossing times based on these friction values. The final output will provide you with a simplified, continent-level friction map that helps estimate border crossing times across the globe.

---


### Code

In [ ]:

#Code to turn the country borders from a vector to a raster
processing.run("gdal:rasterize", {'INPUT':'C:/Users/samja/Downloads/ne_50m_admin_0_boundary_lines_land/ne_50m_admin_0_boundary_lines_land.shp','FIELD':'','BURN':1,'USE_Z':False,'UNITS':1,'WIDTH':0.008333,'HEIGHT':0.008333,'EXTENT':'-115.967578128,139.498377155,-56.294072926,93.400704263 [EPSG:4326]','NODATA':0,'OPTIONS':'','DATA_TYPE':5,'INIT':None,'INVERT':False,'EXTRA':'','OUTPUT':'C:/Users/samja/Downloads/gudie_raster_borders.tif'})

#Code to fill in the nodata values
processing.run("native:fillnodata", {'INPUT':'C:/Users/samja/Downloads/gudie_raster_borders.tif','BAND':1,'FILL_VALUE':0,'OUTPUT':'C:/Users/samja/Downloads/guide_border_fill.tif'})

#Code to scale down border value
processing.run("native:rastercalc", {'LAYERS':['C:/Users/samja/Downloads/guide_border_fill.tif'],'EXPRESSION':'("guide_border_fill@1" = 1) * 0.5','EXTENT':None,'CELL_SIZE':None,'CRS':None,'OUTPUT':'TEMPORARY_OUTPUT'})

#Code subtract borders from continent raster so that each border has a unqiue value based on continet
processing.run("native:rastercalc", {'LAYERS':['C:/Users/samja/Documents/Continent_raster.tif','C:/Users/samja/Downloads/guide_border_fill.tif'],'EXPRESSION':'"Continent_raster@1"-"guide_border_fill@1"','EXTENT':None,'CELL_SIZE':None,'CRS':None,'OUTPUT':'TEMPORARY_OUTPUT'})

#Code to assign friction values to the borders based on the continent
processing.run("native:rastercalc", {'LAYERS':['C:/Users/samja/Downloads/guide_border_fill.tif'],'EXPRESSION':'("continent_border_diff" = 0.5) * 0.115 + ("continent_border_diff" = 1.5) * 0.075 + ("continent_border_diff" = 2.5) * 0.075 + ("continent_border_diff" = 3.5) * 0.06 + ("continent_border_diff" = 4.5) * 0.075 + ("continent_border_diff" = 5.5) * 0.055 + ("continent_border_diff" = 6.5) * 0.075 +("continent_border_diff" = 7.5) * 0.035','EXTENT':None,'CELL_SIZE':None,'CRS':None,'OUTPUT':'TEMPORARY_OUTPUT'})

---

## 2. Shipping lane friction calculation

### Step 1: Open Shapefiles
- **Objective**: Load the shapefile for shipping lanes in QGIS.
  - Open QGIS, then use the **'Add Vector Layer'** tool to load the shapefile containing the country borders.

### Step 2: Reproject the Shipping Lane Raster
- **Objective**: Reproject the shipping lane raster layer into the WGS 84 projection (EPSG:4326).
  - Open the **'Warp (Reproject)'** tool from the Processing Toolbox.
  - In the **Input Layer** field, select the shipping lane raster.
  - In the **Target CRS**, choose **EPSG:4326 - WGS 84** to ensure consistency with the Mercator projection you're using.
  - Run the following code:
    ```
    processing.run("gdal:warpreproject", {'INPUT':'/vsizip/C:/Users/samja/Downloads/raw_2013_shipping_mol_20150714094045.zip/shipping.tif','SOURCE_CRS':QgsCoordinateReferenceSystem('ESRI:54009'),'TARGET_CRS':QgsCoordinateReferenceSystem('EPSG:4326'),'RESAMPLING':0,'NODATA':None,'TARGET_RESOLUTION':None,'OPTIONS':'','DATA_TYPE':0,'TARGET_EXTENT':None,'TARGET_EXTENT_CRS':None,'MULTITHREADING':False,'EXTRA':'','OUTPUT':'C:/Users/samja/OneDrive/Documents/shipping_reprojection.tif'})
    ```

### Step 3: Binarize the Shipping Lane Raster
- **Objective**: Binarize the shipping lanes so that only shipping lanes that receive above a threshold amount of traffic show up.
  - Open the **'Raster Calculator'** tool from the Processing Toolbox.
  - In the **Input Layer** field, select the reprojected shipping lane raster.
  - Enter the following expression:
    ```
    ("shipping@1" < 0.2) * 1 + ("shipping@1" >= 0.2) * 0
    ```

### Step 4: Assign a Friction Value
- **Objective**: Assign a value to the shipping lanes that represents the cost to travel.
  - Open the **'Raster Calculator'** tool from the Processing Toolbox.
  - In the **Input Layer** field, select the binarized shipping lane raster.
  - Enter the following expression:
    ```
    ("shipping@1" < 0) * 0 + ("shipping@1" >= 1) * 0.02
    ```




---

### Code

In [ ]:
#Code to reproject
processing.run("gdal:warpreproject", {'INPUT':'/vsizip/C:/Users/samja/Downloads/raw_2013_shipping_mol_20150714094045.zip/shipping.tif','SOURCE_CRS':QgsCoordinateReferenceSystem('ESRI:54009'),'TARGET_CRS':QgsCoordinateReferenceSystem('EPSG:4326'),'RESAMPLING':0,'NODATA':None,'TARGET_RESOLUTION':None,'OPTIONS':'','DATA_TYPE':0,'TARGET_EXTENT':None,'TARGET_EXTENT_CRS':None,'MULTITHREADING':False,'EXTRA':'','OUTPUT':'C:/Users/samja/OneDrive/Documents/shipping_reprojection.tif'})

# Code to binarize shipping
processing.run("native:rastercalc", {'LAYERS':['C:/Users/samja/Downloads/zv735jp7944/data_EPSG_4326/shipping.tif'],'EXPRESSION':'"shipping@1" >= 0.4 * 1 +"shipping@1" < 0.4 * 1','EXTENT':None,'CELL_SIZE':None,'CRS':None,'OUTPUT':'C:/Users/samja/Downloads/Shipping_binary.tif'})
# Code to assign friction values to shipping
processing.run("native:rastercalc", {'LAYERS':['C:/Users/samja/Downloads/zv735jp7944/data_EPSG_4326/shipping.tif'],'EXPRESSION':'"shipping@1" >= 0.4 * 1 +"shipping@1" < 0.4 * 1','EXTENT':None,'CELL_SIZE':None,'CRS':None,'OUTPUT':'C:/Users/samja/Downloads/Shipping_binary.tif'})

---

## 3. Upload and Reproject Road Network, Railway, and Navigable Rivers



### Step 1: Open Shapefiles
- **Objective**: Load the shapefiles into QGIS.
    - Open QGIS, then use the **'Add Vector Layer'** tool to load the shapefile containing the country borders.

### Step 2: Reproject the Road Network, Railway, and Navigable Rivers Vectors
- **Objective**: Reproject the Road Network, Railway, and Navigable Rivers vector layers into the WGS 84 projection (EPSG:4326).
    - Open the **'Reproject Layer'** tool from the Processing Toolbox.
    - In the **Input Layer** field, select the layer you intend to reproject.
    - In the **Target CRS**, choose **EPSG:4326 - WGS 84** to ensure consistency with the Mercator projection you're using.
    - Click ‘Run’.
    - Repeat for all three layers.


---
---

# **Part 4: Combining all processed rasters into one friction layer**

---
---

# **Part 5: Creating Centroids to Represent Urban Centers**

### Step 1: Open Urban Zone Shapefiles
- Open QGIS, then use the **'Add Vector Layer'** tool to load the shapefile containing discrete urbanization categories.

### Step 2: Binarize Urban Zones
- **Objective**: Extract urban centers from population density data.
  - Open the **'Raster Calculator'** tool from the Processing Toolbox.
  - In the **Input Layer** field, select the urbanization raster.

### Step 3: Vectorize the Raster
- **Objective**: Convert the binarized urban zones into vector format for further processing.
  - Open the **'Polygonize (Raster to Vector)'** tool from the Processing Toolbox.
  - In the **Input Layer** field, select the binarized urbanization raster.

### Step 4: Calculate Centroids
- **Objective**: Determine the centroids of the urban zones to represent urban centers.
  - Open the **'Centroids'** tool from the Processing Toolbox.

### Step 5: Create the Cost Distance Surface
- **Objective**: Generate a cost distance surface using the combined friction layer and urban centroids.
  - Install the r.grass extension.
  - Open the **'r.cost'** tool from the Processing Toolbox.
  - Enter the combined friction layer under the **'Unit Cost Layer'** input.
  - Select the centroids as start points.
  - Run the tool.








----

In [ ]:
#running the vectorize function
processing.run("gdal:polygonize", {'INPUT':'C:/Users/samja/Documents/Only_cities.tif','BAND':1,'FIELD':'DN','EIGHT_CONNECTEDNESS':False,'EXTRA':'','OUTPUT':'C:/Users/samja/Documents/only_city_vectorize.gpkg'})
#running the centroid function
processing.run("native:centroids", {'INPUT':'C:/Users/samja/Documents/only_city_vectorize.gpkg','ALL_PARTS':True,'OUTPUT':'C:/Users/samja/OneDrive/Documents/only_cities_centroids.gpkg'})
#running the cost distance function
processing.run("grass7:r.cost", {'input':'C:/Users/samja/Documents/12-2-combinded-cropped.tif','start_coordinates':None,'stop_coordinates':None,'-k':False,'-n':True,'start_points':'C:/Users/samja/Downloads/ne_110m_populated_places_simple/ne_110m_populated_places_simple.shp','stop_points':None,'start_raster':None,'max_cost':None,'null_cost':None,'memory':5000,'output':'C:/Users/samja/Documents/cumulat_cost.tif','nearest':'C:/Users/samja/Documents/alloca_map.tif','outdir':'C:/Users/samja/Documents/moveme_dir.tif','GRASS_REGION_PARAMETER':None,'GRASS_REGION_CELLSIZE_PARAMETER':0,'GRASS_RASTER_FORMAT_OPT':'','GRASS_RASTER_FORMAT_META':'','GRASS_SNAP_TOLERANCE_PARAMETER':-1,'GRASS_MIN_AREA_PARAMETER':0.0001})

---
---
---